# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [150]:
import pandas as pd
import numpy as np
!pip install -U pandasql

Requirement already up-to-date: pandasql in /Users/dhruvragunathan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages (0.7.3)


In [151]:
#Your code here

homepage_data = pd.read_csv('homepage_actions.csv')

homepage_data


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [152]:
homepage_data.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [153]:
homepage_data.loc[(homepage_data["group"] != "control") | (homepage_data["group"] != "experiment")]

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [154]:
homepage_data[homepage_data['id'] == 536734]

,timestamp,id,group,action
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [155]:
homepage_data.nunique()

# There are 6328 total unique users


timestamp    8188
id           6328
group           2
action          2
dtype: int64

In [156]:
from pandasql import sqldf
import pandas as pd
from sklearn import datasets

In [157]:
q = """SELECT * FROM homepage_data
where id = 182994
"""

sqldf(q, locals())

,timestamp,id,group,action
0,2016-12-14 13:19:00.940677,182994,control,view
1,2016-12-14 13:20:29.564080,182994,control,click


In [158]:
# Remove entries that have potential bad data
# Remove entries where an ID is in both groups
# remove entries where the only action is click

q = """SELECT *,count(*) FROM homepage_data
GROUP BY "id"
having count(*) >1
and COUNT(DISTINCT "group") > 1
"""

sqldf(q, locals())

# No entries where ids are in both groups


,timestamp,id,group,action,count(*)


In [159]:
q = """
SELECT * FROM
homepage_data
where "id" IN
(SELECT id FROM homepage_data
where action = "click"
GROUP BY "action"
having count(*) = 1)
"""

sqldf(q, globals())

# Since no records were returned where click was the only action, we do not have bad data in this scenario

,timestamp,id,group,action


The output above represents bad data since you need to view before you can click. Let's create a new dataframe without this data

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [160]:
import scipy
from scipy import stats

In [161]:
#Your code here

# Null hypothesis. H0 = The experimental homepage was not more effective at generating clicks than the control page.

# Alternate hypothesis = The experimental homepage was more effective at generating clicks than the control page

# Use a chi-squared test to determine if the experimental homepage makes users more likely to click on it

# Our significance level is alpha = 0.05


In [169]:
# Calculate the frequency of click versus not clicking in the control group

q = """
SELECT *
FROM homepage_data
where "group" = "control"
and action = "click"
group by "id"
having count(*) = 1
"""

Ctl_Clks = len(sqldf(q, locals()))

q = """
SELECT *
FROM homepage_data
where "group" = "control"
"""

Ctl_total = len(sqldf(q, locals()))

Control_Freq = [Ctl_Clks, Ctl_total]

Control_Freq

[932, 4264]

In [170]:
q = """
SELECT *
FROM homepage_data
where "group" = "experiment"
group by "id"
having count(*) = 1
"""

len(sqldf(q, locals()))

2068

In [171]:
# Calculate the frequency of click versus not clicking in the experimental group

a = """
SELECT *
FROM homepage_data
where "group" = "experiment"
and action = "click"
group by "id"
having count(*) = 1
"""

Exp_Clks = len(sqldf(a, locals()))

a = """
SELECT *
FROM homepage_data
where "group" = "experiment"
"""

Exp_total = len(sqldf(a, locals()))

Exp_Freq = [Exp_Clks, Exp_total]

Exp_Freq

[928, 3924]

In [172]:
# Conduct the Chi Squared Test between the expected and observed Frequency

scipy.stats.chisquare(Exp_Freq, Control_Freq)


Power_divergenceResult(statistic=27.127861565839165, pvalue=1.904334979585688e-07)

In [ ]:
# The p-value here is nearly 0 and less than our threshold of alpha - 0.05 for statistical significance.
# Therefore, we can reject the null hypothesis in favor of the alternate hypothesis that the experimental webpage is
# more likely to generate clicks.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [175]:
#Your code here

Control_Click_Rate = Control_Freq[0]/Control_Freq[1]

Exp_Experiment_Clk_Rate = Control_Click_Rate * Exp_Freq[1]

Exp_Experiment_Clk_Rate


857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [187]:
#Your code here

std = ((Control_Freq[0])*(Control_Click_Rate)*(1-Control_Click_Rate))**0.5

z_score = (Exp_Freq[0]-Exp_Experiment_Clk_Rate)/std

z_score


5.5731147486889565

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [188]:
#Your code here

scipy.stats.norm.sf(abs(z_score))


1.251123168406923e-08

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes the analysis roughly matches that of the previous statistical test. The p-value here is approximately 0 similarly to the chi-squared test.



## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.